# Text Summarization

In [1]:
import os
import sys
import requests
import pandas as pd
import re
print(sys.version)

3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]


In [2]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

#### Copy files to local FS from GCP bucket

In [3]:
def get_gcs_data (bucket_name, folder_name, file_name, path_local):
    url = 'https://storage.googleapis.com/' + bucket_name + '/' + folder_name + '/' + file_name
    r = requests.get(url)
    open(path_local + '/' + file_name , 'wb').write(r.content)

In [4]:
bucket_name = 'msca-bdp-data-open'
folder_name = 'books'
file_name = ['3boat10.txt']
path_local = '/home/jupyter/data/books'

os.makedirs(path_local, exist_ok=True)

for file in file_name:
    get_gcs_data (bucket_name = bucket_name,
                 folder_name = folder_name,
                 file_name = file,
                 path_local = path_local)
    print('Downloaded: ' + file)

Downloaded: 3boat10.txt


In [5]:
bucket_name = 'msca-bdp-data-open'
folder_name = 'news'
file_name = ['news_toyota.json']
path_local = '/home/jupyter/data/news'

os.makedirs(path_local, exist_ok=True)

for file in file_name:
    get_gcs_data (bucket_name = bucket_name,
                 folder_name = folder_name,
                 file_name = file,
                 path_local = path_local)
    print('Downloaded: ' + file)

Downloaded: news_toyota.json


## Summarization with Summa

In [6]:
# !pip install summa

In [7]:
import summa

In [8]:
text = '''

The University of Chicago is an urban research university that has driven new ways of thinking since 1890. Our commitment to free and open inquiry draws inspired scholars to our global campuses, where ideas are born that challenge and change the world.

We empower individuals to challenge conventional thinking in pursuit of original ideas. Students in the College develop critical, analytic, and writing skills in our rigorous, interdisciplinary core curriculum. Through graduate programs, students test their ideas with UChicago scholars, and become the next generation of leaders in academia, industry, nonprofits, and government.

UChicago research has led to such breakthroughs as discovering the link between cancer and genetics, establishing revolutionary theories of economics, and developing tools to produce reliably excellent urban schooling. We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Laboratory, Fermi National Accelerator Laboratory, and the Marine Biological Laboratory in Woods Hole, Massachusetts.

The University of Chicago is enriched by the city we call home. In partnership with our neighbors, we invest in Chicago's mid-South Side across such areas as health, education, economic growth, and the arts. Together with our medical center, we are the largest private employer on the South Side.

In all we do, we are driven to dig deeper, push further, and ask bigger questions—and to leverage our knowledge to enrich all human life. Our diverse and creative students and alumni drive innovation, lead international conversations, and make masterpieces. Alumni and faculty, lecturers and postdocs go on to become Nobel laureates, CEOs, university presidents, attorneys general, literary giants, and astronauts. 
'''

print(text)



The University of Chicago is an urban research university that has driven new ways of thinking since 1890. Our commitment to free and open inquiry draws inspired scholars to our global campuses, where ideas are born that challenge and change the world.

We empower individuals to challenge conventional thinking in pursuit of original ideas. Students in the College develop critical, analytic, and writing skills in our rigorous, interdisciplinary core curriculum. Through graduate programs, students test their ideas with UChicago scholars, and become the next generation of leaders in academia, industry, nonprofits, and government.

UChicago research has led to such breakthroughs as discovering the link between cancer and genetics, establishing revolutionary theories of economics, and developing tools to produce reliably excellent urban schooling. We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Labo

In [9]:
textSum = summa.summarizer.summarize(text)
print(textSum)

The University of Chicago is an urban research university that has driven new ways of thinking since 1890.
Through graduate programs, students test their ideas with UChicago scholars, and become the next generation of leaders in academia, industry, nonprofits, and government.


#### Summa keyword extraction

In [10]:
keywords = summa.keywords.keywords(text)
print(keywords)

new
laboratories
laboratory
students
develop
developing
national
deeper
private
bigger
industry
conventional
hole
uchicago
urban research
university
interdisciplinary core
education economic
economics
establishing


### Summarizing book with Summa

In [11]:
directory = '/home/jupyter/data/books/'
book = '3boat10.txt'
f = open(directory+book)
textRaw = f.read()
text = re.sub(r'\n', ' ', textRaw)

In [12]:
%time textSum = summa.summarizer.summarize(text, ratio=0.001)
print(textSum)

CPU times: user 2min 5s, sys: 42.9 s, total: 2min 48s
Wall time: 1min 28s
And we sit there, by its margin, while the moon, who loves it too, stoops  down to kiss it with a sister's kiss, and throws her silver arms around  it clingingly; and we watch it as it flows, ever singing, ever  whispering, out to meet its king, the sea - till our voices die away in  silence, and the pipes go out - till we, common-place, everyday young men  enough, feel strangely full of thoughts, half sad, half sweet, and do not  care or want to speak - till we laugh, and, rising, knock the ashes from  our burnt-out pipes, and say "Good-night," and, lulled by the lapping  water and the rustling trees, we fall asleep beneath the great, still  stars, and dream that the world is young again - young and sweet as she  used to be ere the centuries of fret and care had furrowed her fair face,  ere her children's sins and follies had made old her loving heart - sweet  as she was in those bygone days when, a new-made mot

In [13]:
%time textSum = summa.summarizer.summarize(text, words=100)
print(textSum)

CPU times: user 2min 5s, sys: 42.3 s, total: 2min 47s
Wall time: 1min 30s



### Summarizing news articles with Summa

In [14]:
directory = '/home/jupyter/data/news/'
news_articles = 'news_toyota.json'

path = directory+news_articles

In [15]:
news_df = pd.read_json(directory+news_articles, orient='records', lines=True)

In [16]:
# Filter non-English news
news_eng = news_df[news_df['language']=='english'].reset_index(drop=True)

In [17]:
# Remove /n characters to avoid problems with analysis
news_eng['text_clean'] = news_eng['text'].map(lambda x: re.sub(r'\n', '.  ', str(x)))

In [18]:
pd.set_option('display.max_colwidth', 100)
news_eng[['text', 'text_clean']].head(5)

,text,text_clean
0,"QR Code Link to This Post All maintenance receipts available, one owner truck. Cash sale. No tra...","QR Code Link to This Post All maintenance receipts available, one owner truck. Cash sale. No tra..."
1,"0 \nNEW YORK: Automakers reported mixed US car sales in January, with strong demand for SUVs and...","0 . NEW YORK: Automakers reported mixed US car sales in January, with strong demand for SUVs an..."
2,transmission: automatic 2005 Toyota Camry LE 4 door 4 cyl AUTOMATIC VERY CLEAN INSIDE CLOTH IN...,transmission: automatic 2005 Toyota Camry LE 4 door 4 cyl AUTOMATIC VERY CLEAN INSIDE CLOTH IN...
3,favorite this post Brand New Toyota Avalon Floor Mats - $115 (New Britain) hide this posting unh...,favorite this post Brand New Toyota Avalon Floor Mats - $115 (New Britain) hide this posting unh...
4,more ads by this user QR Code Link to This Post Black w/Piano Black w/Perforated NuLuxe Seat Tri...,more ads by this user QR Code Link to This Post Black w/Piano Black w/Perforated NuLuxe Seat Tri...


In [19]:
text = str(news_eng['text_clean'].tolist())
#text = str(news_eng['text'].tolist())
text[:1000]

"['QR Code Link to This Post All maintenance receipts available, one owner truck. Cash sale. No trades.   6477478013', '0 .  NEW YORK: Automakers reported mixed US car sales in January, with strong demand for SUVs and pickup trucks continuing to provide a cushion in a declining overall auto market. .  Ford and Fiat Chrysler reported declines in year-over-year sales, while General Motors scored a modest increase and Toyota saw a more substantial jump. .  US car sales fell last year for the first time since the financial crisis and are projected to decline again in 2018. Still, analysts and industry executives expect US sales this year to come in above a solid 16 million vehicles amid low unemployment and strong consumer confidence. .  “US economic factors are very healthy and we’re seeing the effect in the auto industry — not just in strong demand for SUVs and pickups, but in demand for high trim versions of vehicles,” said Mark LaNeve, Ford’s vice president for US marketing. .  Ford’s 

In [20]:
%time textSum = summa.summarizer.summarize(text, ratio=0.003)
print(textSum)

CPU times: user 17.6 s, sys: 5.74 s, total: 23.4 s
Wall time: 15 s
That works out to a 32.7 per cent share of the market, and signals the company’s first sales uptick in three years.', 'January Auto Sales Winners and Losers Automobile 3 days ago Ed Tahaney © Automobile Magazine Staff January 2018 auto sales are looking up and Toyota was the biggest winner with a 17 percent gain over last year\'s sales figures.
In Japan, selling at least 1,000 fuel cell vehicles per month (well in excess of 10,000 per year) Beginning sales of fuel cell buses in small numbers by early 2017, focusing on Tokyo; preparing to sell over 100 fuel cell buses ahead of the Olympic and Paralympic Games Tokyo 2020 Achieving sales of 1.5 million hybrids annually and 15 million hybrids cumulatively by 2020 Reducing average CO 2 emissions from new vehicles by more than 22 percent by 2020 (compared to Toyota\'s 2010 global average) Toyota Electrified Vehicle Sales Based on TMC data (Unit = 1,000 vehicles) ', '– Digital

In [21]:
from textblob import TextBlob
blob = TextBlob(textSum)

blob.sentences

[Sentence("That works out to a 32.7 per cent share of the market, and signals the company’s first sales uptick in three years."),
 Sentence("', 'January Auto Sales Winners and Losers Automobile 3 days ago Ed Tahaney © Automobile Magazine Staff January 2018 auto sales are looking up and Toyota was the biggest winner with a 17 percent gain over last year\'s sales figures."),
 Sentence("In Japan, selling at least 1,000 fuel cell vehicles per month (well in excess of 10,000 per year) Beginning sales of fuel cell buses in small numbers by early 2017, focusing on Tokyo; preparing to sell over 100 fuel cell buses ahead of the Olympic and Paralympic Games Tokyo 2020 Achieving sales of 1.5 million hybrids annually and 15 million hybrids cumulatively by 2020 Reducing average CO 2 emissions from new vehicles by more than 22 percent by 2020 (compared to Toyota\'s 2010 global average) Toyota Electrified Vehicle Sales Based on TMC data (Unit = 1,000 vehicles) ', '– Digital Producer, Birmingham Busin

## Summarization with sumy LexRank

In [22]:
# !pip install sumy

In [23]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer 

In [24]:
text = '''

The University of Chicago is an urban research university that has driven new ways of thinking since 1890. Our commitment to free and open inquiry draws inspired scholars to our global campuses, where ideas are born that challenge and change the world.

We empower individuals to challenge conventional thinking in pursuit of original ideas. Students in the College develop critical, analytic, and writing skills in our rigorous, interdisciplinary core curriculum. Through graduate programs, students test their ideas with UChicago scholars, and become the next generation of leaders in academia, industry, nonprofits, and government.

UChicago research has led to such breakthroughs as discovering the link between cancer and genetics, establishing revolutionary theories of economics, and developing tools to produce reliably excellent urban schooling. We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Laboratory, Fermi National Accelerator Laboratory, and the Marine Biological Laboratory in Woods Hole, Massachusetts.

The University of Chicago is enriched by the city we call home. In partnership with our neighbors, we invest in Chicago's mid-South Side across such areas as health, education, economic growth, and the arts. Together with our medical center, we are the largest private employer on the South Side.

In all we do, we are driven to dig deeper, push further, and ask bigger questions—and to leverage our knowledge to enrich all human life. Our diverse and creative students and alumni drive innovation, lead international conversations, and make masterpieces. Alumni and faculty, lecturers and postdocs go on to become Nobel laureates, CEOs, university presidents, attorneys general, literary giants, and astronauts. 
'''

In [25]:
parser = PlaintextParser.from_string(text, Tokenizer('english'))

In [26]:
summarizer = LexRankSummarizer()

#Summarize the document, output 2 sentences
%time summary = summarizer(parser.document, 2)

for sentence in summary:
    print(sentence)

CPU times: user 12 ms, sys: 34 µs, total: 12.1 ms
Wall time: 11.7 ms
The University of Chicago is an urban research university that has driven new ways of thinking since 1890.
Our commitment to free and open inquiry draws inspired scholars to our global campuses, where ideas are born that challenge and change the world.


### Summarizing book with LexRank

In [27]:
directory = '/home/jupyter/data/books/'
book = '3boat10.txt'
text = directory+book

In [28]:
parser = PlaintextParser.from_file(text, Tokenizer('english'))

In [29]:
summarizer = LexRankSummarizer()

%time summary = summarizer(parser.document, 10)

for sentence in summary:
    print(sentence)

CPU times: user 4min 15s, sys: 458 ms, total: 4min 15s
Wall time: 4min 15s
George said he should be all right, and would rather like it, but he would advise Harris and me not to think of it, as he felt sure we should both be ill. Harris said that, to himself, it was always a mystery how people managed to get sick at sea - said he thought people must do it on purpose, from affectation - said he had often wished to be, but had never been able.
We would find the hammer for him, and then he would have lost sight of the mark he had made on the wall, where the nail was to go in, and each of us had to get up on the chair, beside him, and see if we could find it; and we would each discover it in a different place, and he would call us all fools, one after another, and tell us to get down.
I was in Liverpool at the time, and my friend said that if I didn't mind he would get me to take them back with me to London, as he should not be coming up for a day or two himself, and he did not think the c

In [30]:
summarizer = LexRankSummarizer()

%time summary = summarizer(parser.document, 20)

for sentence in summary:
    print(sentence)

CPU times: user 4min 17s, sys: 766 ms, total: 4min 17s
Wall time: 4min 17s
I had them all.
He said:
George said he should be all right, and would rather like it, but he would advise Harris and me not to think of it, as he felt sure we should both be ill. Harris said that, to himself, it was always a mystery how people managed to get sick at sea - said he thought people must do it on purpose, from affectation - said he had often wished to be, but had never been able.
"There you are!
We would find the hammer for him, and then he would have lost sight of the mark he had made on the wall, where the nail was to go in, and each of us had to get up on the chair, beside him, and see if we could find it; and we would each discover it in a different place, and he would call us all fools, one after another, and tell us to get down.
I was in Liverpool at the time, and my friend said that if I didn't mind he would get me to take them back with me to London, as he should not be coming up for a day o

In [31]:
summarizer = LexRankSummarizer()

%time summary = summarizer(parser.document, 30)

for sentence in summary:
    print(sentence)

CPU times: user 4min 15s, sys: 671 ms, total: 4min 16s
Wall time: 4min 15s
I had them all.
He said:
George said he should be all right, and would rather like it, but he would advise Harris and me not to think of it, as he felt sure we should both be ill. Harris said that, to himself, it was always a mystery how people managed to get sick at sea - said he thought people must do it on purpose, from affectation - said he had often wished to be, but had never been able.
"There you are!
We would find the hammer for him, and then he would have lost sight of the mark he had made on the wall, where the nail was to go in, and each of us had to get up on the chair, beside him, and see if we could find it; and we would each discover it in a different place, and he would call us all fools, one after another, and tell us to get down.
"Why, some people would have had a man in to do a little thing like that!"
I was in Liverpool at the time, and my friend said that if I didn't mind he would get me to 

## Summarization with PyTextRank

In [32]:
# !pip install pytextrank

In [33]:
import pytextrank
import spacy

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("textrank")

In [34]:
text = '''

The University of Chicago is an urban research university that has driven new ways of thinking since 1890. Our commitment to free and open inquiry draws inspired scholars to our global campuses, where ideas are born that challenge and change the world.

We empower individuals to challenge conventional thinking in pursuit of original ideas. Students in the College develop critical, analytic, and writing skills in our rigorous, interdisciplinary core curriculum. Through graduate programs, students test their ideas with UChicago scholars, and become the next generation of leaders in academia, industry, nonprofits, and government.

UChicago research has led to such breakthroughs as discovering the link between cancer and genetics, establishing revolutionary theories of economics, and developing tools to produce reliably excellent urban schooling. We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Laboratory, Fermi National Accelerator Laboratory, and the Marine Biological Laboratory in Woods Hole, Massachusetts.

The University of Chicago is enriched by the city we call home. In partnership with our neighbors, we invest in Chicago's mid-South Side across such areas as health, education, economic growth, and the arts. Together with our medical center, we are the largest private employer on the South Side.

In all we do, we are driven to dig deeper, push further, and ask bigger questions—and to leverage our knowledge to enrich all human life. Our diverse and creative students and alumni drive innovation, lead international conversations, and make masterpieces. Alumni and faculty, lecturers and postdocs go on to become Nobel laureates, CEOs, university presidents, attorneys general, literary giants, and astronauts. 
'''

In [35]:
print(text)



The University of Chicago is an urban research university that has driven new ways of thinking since 1890. Our commitment to free and open inquiry draws inspired scholars to our global campuses, where ideas are born that challenge and change the world.

We empower individuals to challenge conventional thinking in pursuit of original ideas. Students in the College develop critical, analytic, and writing skills in our rigorous, interdisciplinary core curriculum. Through graduate programs, students test their ideas with UChicago scholars, and become the next generation of leaders in academia, industry, nonprofits, and government.

UChicago research has led to such breakthroughs as discovering the link between cancer and genetics, establishing revolutionary theories of economics, and developing tools to produce reliably excellent urban schooling. We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Labo

In [36]:
phrases = 1 # Summarize based on its top N phrases
sentences = 1 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Laboratory, Fermi National Accelerator Laboratory, and the Marine Biological Laboratory in Woods Hole, Massachusetts.


In [37]:
phrases = 2 # Summarize based on its top N phrases
sentences = 2 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Laboratory, Fermi National Accelerator Laboratory, and the Marine Biological Laboratory in Woods Hole, Massachusetts.


The University of Chicago is an urban research university that has driven new ways of thinking since 1890.


In [38]:
phrases = 1 # Summarize based on its top N phrases
sentences = 3 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Laboratory, Fermi National Accelerator Laboratory, and the Marine Biological Laboratory in Woods Hole, Massachusetts.


The University of Chicago is an urban research university that has driven new ways of thinking since 1890.
Our commitment to free and open inquiry draws inspired scholars to our global campuses, where ideas are born that challenge and change the world.


In [39]:
phrases = 10 # Summarize based on its top N phrases
sentences = 3 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Laboratory, Fermi National Accelerator Laboratory, and the Marine Biological Laboratory in Woods Hole, Massachusetts.
Alumni and faculty, lecturers and postdocs go on to become Nobel laureates, CEOs, university presidents, attorneys general, literary giants, and astronauts.


We empower individuals to challenge conventional thinking in pursuit of original ideas.


In [40]:
phrases = 15 # Summarize based on its top N phrases
sentences = 5 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

We generate new insights for the benefit of present and future generations with our national and affiliated laboratories: Argonne National Laboratory, Fermi National Accelerator Laboratory, and the Marine Biological Laboratory in Woods Hole, Massachusetts.
Alumni and faculty, lecturers and postdocs go on to become Nobel laureates, CEOs, university presidents, attorneys general, literary giants, and astronauts.


We empower individuals to challenge conventional thinking in pursuit of original ideas.
Through graduate programs, students test their ideas with UChicago scholars, and become the next generation of leaders in academia, industry, nonprofits, and government.
In partnership with our neighbors, we invest in Chicago's mid-South Side across such areas as health, education, economic growth, and the arts.


### Summarizing book with PyTextRank

In [41]:
directory = '/home/jupyter/data/books/'
book = '3boat10.txt'
f = open(directory+book)
textRaw = f.read()
text = re.sub(r'\n', ' ', textRaw)

In [42]:
%%time

phrases = 15 # Summarize based on its top N phrases
sentences = 5 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

 The expression on the face of the  man who, with his hands in his pockets, stands by the stern, smoking a  cigar, is sufficient to excuse a breach of the peace by itself; and the  lordly whistle for you to get out of the way would, I am confident,  ensure a verdict of "justifiable homicide" from any jury of river men.
*  You meet other boats there,  and river gossip is exchanged.
 "Go away, you miserable old man," I said.
 I was thinking of other things,  and forgot, as any one might easily understand, that I was steering, and  the consequence was that we had got mixed up a good deal with the tow- path.
 George and I - I don't know what had become of Harris; he had gone out  and had a shave, early in the afternoon, and had then come back and spent  full forty minutes in pipeclaying his shoes, we had not seen him since -  George and I, therefore, and the dog, left to ourselves, went for a walk  to Wallingford on the second evening, and, coming home, we called in at a  little river-side

In [43]:
%%time

phrases = 15 # Summarize based on its top N phrases
sentences = 20 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

 The expression on the face of the  man who, with his hands in his pockets, stands by the stern, smoking a  cigar, is sufficient to excuse a breach of the peace by itself; and the  lordly whistle for you to get out of the way would, I am confident,  ensure a verdict of "justifiable homicide" from any jury of river men.
*  You meet other boats there,  and river gossip is exchanged.
 "Go away, you miserable old man," I said.
 I was thinking of other things,  and forgot, as any one might easily understand, that I was steering, and  the consequence was that we had got mixed up a good deal with the tow- path.
 George and I - I don't know what had become of Harris; he had gone out  and had a shave, early in the afternoon, and had then come back and spent  full forty minutes in pipeclaying his shoes, we had not seen him since -  George and I, therefore, and the dog, left to ourselves, went for a walk  to Wallingford on the second evening, and, coming home, we called in at a  little river-side

In [44]:
%%time

phrases = 50 # Summarize based on its top N phrases
sentences = 20 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

 George and I - I don't know what had become of Harris; he had gone out  and had a shave, early in the afternoon, and had then come back and spent  full forty minutes in pipeclaying his shoes, we had not seen him since -  George and I, therefore, and the dog, left to ourselves, went for a walk  to Wallingford on the second evening, and, coming home, we called in at a  little river-side inn, for a rest, and other things.
 He came and sat down on things, just when they were wanted to be packed;  and he laboured under the fixed belief that, whenever Harris or George  reached out their hand for anything, it was his cold, damp nose that they  wanted.
 For other breakfast things, George suggested eggs and bacon, which were  easy to cook, cold meat, tea, bread and butter, and jam.
 Harris will be just that sort of man when he grows up, I know, and I told  him so.
 Harris told them they ought to be grateful for a little excitement,  sitting there fishing all day, and he also said that he was s

### Summarizing news articles with PyTextRank

In [45]:
directory = '/home/jupyter/data/news/'
news_articles = 'news_toyota.json'

path = directory+news_articles

In [46]:
news_df = pd.read_json(directory+news_articles, orient='records', lines=True)

In [47]:
# Filter non-English news
news_eng = news_df[news_df['language']=='english'].reset_index(drop=True)

In [48]:
# Remove /n characters to avoid problems with analysis
news_eng['text_clean'] = news_eng['text'].map(lambda x: re.sub(r'\n', '.  ', str(x)))

In [49]:
pd.set_option('display.max_colwidth', 100)
news_eng[['text', 'text_clean']].head(5)

,text,text_clean
0,"QR Code Link to This Post All maintenance receipts available, one owner truck. Cash sale. No tra...","QR Code Link to This Post All maintenance receipts available, one owner truck. Cash sale. No tra..."
1,"0 \nNEW YORK: Automakers reported mixed US car sales in January, with strong demand for SUVs and...","0 . NEW YORK: Automakers reported mixed US car sales in January, with strong demand for SUVs an..."
2,transmission: automatic 2005 Toyota Camry LE 4 door 4 cyl AUTOMATIC VERY CLEAN INSIDE CLOTH IN...,transmission: automatic 2005 Toyota Camry LE 4 door 4 cyl AUTOMATIC VERY CLEAN INSIDE CLOTH IN...
3,favorite this post Brand New Toyota Avalon Floor Mats - $115 (New Britain) hide this posting unh...,favorite this post Brand New Toyota Avalon Floor Mats - $115 (New Britain) hide this posting unh...
4,more ads by this user QR Code Link to This Post Black w/Piano Black w/Perforated NuLuxe Seat Tri...,more ads by this user QR Code Link to This Post Black w/Piano Black w/Perforated NuLuxe Seat Tri...


In [50]:
text = str(news_eng['text_clean'].tolist())
#text = str(news_eng['text'].tolist())
text[:1000]

"['QR Code Link to This Post All maintenance receipts available, one owner truck. Cash sale. No trades.   6477478013', '0 .  NEW YORK: Automakers reported mixed US car sales in January, with strong demand for SUVs and pickup trucks continuing to provide a cushion in a declining overall auto market. .  Ford and Fiat Chrysler reported declines in year-over-year sales, while General Motors scored a modest increase and Toyota saw a more substantial jump. .  US car sales fell last year for the first time since the financial crisis and are projected to decline again in 2018. Still, analysts and industry executives expect US sales this year to come in above a solid 16 million vehicles amid low unemployment and strong consumer confidence. .  “US economic factors are very healthy and we’re seeing the effect in the auto industry — not just in strong demand for SUVs and pickups, but in demand for high trim versions of vehicles,” said Mark LaNeve, Ford’s vice president for US marketing. .  Ford’s 

In [51]:
%%time

phrases = 15 # Summarize based on its top N phrases
sentences = 5 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

 TMMC builds 571,535 vehicles in 2017 to lead all Canadian automotive manufacturers 40% of Toyota vehicles sold in Canada are made in Canada , TMMC is TCI\'s primary supplier TMMC once again a net exporter of vehicles, making a significant contribution to our economy.
Every year, the skills and passion of more than 8,000 Canadians build Toyota vehicles at three assembly plants in Cambridge and Woodstock , supported by thousands of people across the country who supply parts and services..  "We have a very strong partnership with TMMC and its Team Members, and each Toyota vehicle manufactured delivers the quality, dependability and reliability that our Canadian customers have come to expect," Hutchinson added.
do NOT contact me with unsolicited services or offers post id: 6481378985', 'by Autocar Pro News Desk Feb 01, 2018 The Fortuner, Innova and Corolla Altis are powering Toyota sales. .
Toyota also aims to further expand sales of hybrid electric vehicles in emerging markets to further

In [52]:
%%time

phrases = 15 # Summarize based on its top N phrases
sentences = 20 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

 TMMC builds 571,535 vehicles in 2017 to lead all Canadian automotive manufacturers 40% of Toyota vehicles sold in Canada are made in Canada , TMMC is TCI\'s primary supplier TMMC once again a net exporter of vehicles, making a significant contribution to our economy.
Every year, the skills and passion of more than 8,000 Canadians build Toyota vehicles at three assembly plants in Cambridge and Woodstock , supported by thousands of people across the country who supply parts and services..  "We have a very strong partnership with TMMC and its Team Members, and each Toyota vehicle manufactured delivers the quality, dependability and reliability that our Canadian customers have come to expect," Hutchinson added.
do NOT contact me with unsolicited services or offers post id: 6481378985', 'by Autocar Pro News Desk Feb 01, 2018 The Fortuner, Innova and Corolla Altis are powering Toyota sales. .
Toyota also aims to further expand sales of hybrid electric vehicles in emerging markets to further

In [53]:
%%time

phrases = 50 # Summarize based on its top N phrases
sentences = 20 # Yielding its top K sentences

doc = nlp(text)
tr = doc._.textrank

for sent in tr.summary(limit_phrases=phrases,
                       limit_sentences=sentences):
    print(sent)

Having produced more than seven million vehicles, popular Canadian models built at these facilities include Toyota RAV4, Toyota Corolla, Lexus RX 350 and RX 450h hybrid..  SOURCE Toyota Canada Inc..  View original content with multimedia: http://www.newswire.ca/en/releases/archive/February2018/01/c4462.html.
 "We are very proud Toyota was, once again, the number one maker of vehicles in Canada last year," said Larry Hutchinson , President and CEO, Toyota Canada Inc.
 Those two CH-R variants are smallish crossover SUVs that others, most notably Japan’s Honda Motor Co ( 7267.T ), have leveraged to grow sales significantly in China..  A Toyota spokesman said that though the 2018 sales target was not one that can be easily achieved due to the highly competitive market environment, the recent launch of a redesigned Camry sedan and the planned introduction of two subcompact SUVs later this year would enable Toyota to challenge the previous year’s numbers..  China’s overall vehicle market gro

In [54]:
import datetime
import pytz

datetime.datetime.now(pytz.timezone('US/Central')).strftime("%a, %d %B %Y %H:%M:%S")

'Sun, 30 October 2022 12:29:31'